# Sequence Alignments 

It is common to want to align sequences.  BioRuby can do this for you in simple cases:



In [1]:
require 'bio'

seqs = [ 'atgca', 'aagca', 'acgca', 'acgcg' ]
seqs = seqs.collect{ |x| Bio::Sequence::NA.new(x) }
# creates alignment object
a = Bio::Alignment.new(seqs)
puts a.consensus 
puts a.consensus_iupac

a?gc?
ahgcr


Try adding a few nucleotides to one of the sequences.  What happens in the alignment?  Add the same nucleotides to the other sequences, one-by-one... what happens in the alignment?   What do you learn from this?


### more complicated alignments, e.g. with gaps, requires an alignment algorithm.

A common way to do alignments is ClustalW.  This is installed on your virtual machine.  

To access ClustalW from BioRuby's Alignment object, you do this:


In [2]:
require 'bio'

seqs = [ 'ctggtatt', 'aagcatt', 'acgcagtt', 'acgcgtt' ]
seqs = seqs.collect{ |x| Bio::Sequence::NA.new(x) }
# creates alignment object
a = Bio::Alignment.new(seqs)
puts "BioRuby simple alignment"
puts a.consensus_string
puts a.consensus_iupac
puts
puts "ClustalW alignment"
factory = Bio::ClustalW.new
a2 = a.do_align(factory)
puts a2.consensus_string
puts a2.consensus_iupac

BioRuby simple alignment
??g???t?
mhgsddt?

ClustalW alignment
???G???T?
?wvgyrkt?


# Sequence Searching

We will look at sequence searches using FASTA and BLAST.  Both of these have been installed for you.

See docs at http://bioruby.org/rdoc/Bio/Fasta and ....

I have downloaded 160 Ebola virus sequences for you in the "Databases" folder.  You need to concatenate all of the single-sequence FASTA files into a single FASTA file, which is used as the database for a FASTA search:

    $ cat *.fa  >  EBOLA.fa

#  <span style="color:red;">NOTE:  BioRuby's FASTA report parser is broken</span>

<span style="color:red;">The code below WILL NOT WORK right now. However, you can look at the structure of the code to understand what it is supposed to do...</span>

In [ ]:
require 'bio'
require 'stringio'

factory = Bio::Fasta.local('/home/osboxes/fasta-36.3.8d/bin/fasta36', '/home/osboxes/Sequences/EBOLA.fa') 
find = StringIO.new(">myseq\nAGCCCCGTTTTTTGCCACTGGTTATTTAATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT")

ff = Bio::FlatFile.auto(find)   # 'find' is a streamable object

# Iterates over each entry. the variable "entry" is a Bio::FastaFormat object:

ff.each do |entry|
   
  # executes homology search. Returns Bio::Fasta::Report object.
  report = factory.query(entry)

  # Iterates over each hit
  report.each do |hit|
      print "#{hit.query_id} : evalue #{hit.evalue}\t#{hit.target_id} at "
      p hit.lap_at
  end
end

# BLAST searches

to install Blast for yourself:  
    
    $  sudo apt-get install ncbi-blast+
    
    $  sudo apt-get install ncbi-blast+-legacy   (this is required for BioRuby!)
    
### Custom BLAST databases
    
Blast requires:

    A sequence to be used as the query

    A BLAST-formatted database

Common BLAST databases can be downloaded from NCBI:
ftp://ftp.ncbi.nlm.nih.gov/blast/db/ 

We will create our own small database for this class using the EBOLA data.

First, create a new folder "Databases" in your home directory, and **copy** EBOLA.fa into that location.

Now, execute this command:

    $  makeblastdb -in EBOLA.fa -dbtype 'nucl' -out EBOLA

This creates some index files that are used by BLAST to speed-up the search.    
    
The search itself is very similar to the FASTA search above, but a little bit easier because the search query can be passed as a string:

In [3]:
require 'bio'
require 'stringio'

# blast is installed globally, so you don't need to include the full path to blastn
factory = Bio::Blast.local('blastn', './Databases/EBOLA') 

# executes homology search. Returns Bio::Blast::Report object.
report = factory.query(">myseq\nAGCCCCGTTTTTTGCCACTGGAATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT") # this is a Bio::Blast::Report

# Iterates over each hit
report.each do |hit|
  print "#{hit.hit_id} : evalue #{hit.evalue}\t#{hit.target_id} at "
  puts "#{hit.lap_at}"   # this tells you start and end of both the query and the hit sequences
  hit.each do |hsp|
    puts hsp.qseq  # this is the gapped Alignment as text of the query
    puts hsp.hseq  # this is the gapped Alignment as text of the hit
  end
end

gnl|BL_ORD_ID|154 : evalue 9.63642e-21	154 at [1, 64, 17514, 17583]
AGCCCCGTTTTTTGCCACTGG------AATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
AGCCCCGTTTTTTGCCACTGGTTATTTAATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
gnl|BL_ORD_ID|41 : evalue 9.63642e-21	41 at [1, 64, 17514, 17583]
AGCCCCGTTTTTTGCCACTGG------AATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
AGCCCCGTTTTTTGCCACTGGTTATTTAATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
gnl|BL_ORD_ID|40 : evalue 9.63642e-21	40 at [1, 64, 17514, 17583]
AGCCCCGTTTTTTGCCACTGG------AATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
AGCCCCGTTTTTTGCCACTGGTTATTTAATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
gnl|BL_ORD_ID|39 : evalue 9.63642e-21	39 at [1, 64, 17514, 17583]
AGCCCCGTTTTTTGCCACTGG------AATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
AGCCCCGTTTTTTGCCACTGGTTATTTAATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
gnl|BL_ORD_ID|38 : evalue 9.63642e-21	38 at [1, 64, 17514, 17583]
AGCCCCGTTTTTTGCCACTGG------AATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
AGCCCCGTTTTTTGCCACTGGTTATTTAA

AGCCCCGTTTTTTGCCACTGGGTATTTAATTAAGCCAATAACGTCAAGTGCCAGGTCTAGTGAGTGGTAT
gnl|BL_ORD_ID|140 : evalue 5.72664e-16	140 at [1, 64, 17501, 17570]
AGCCCCGTTTTTTGCCACTGG------AATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
AGCCCCGTTTTTTGCCACTGGGTATTTAATTAAGCCAATAACGTCAAGTGCCAGGTCTAGTGAGTGGTAT
gnl|BL_ORD_ID|139 : evalue 5.72664e-16	139 at [1, 64, 17470, 17539]
AGCCCCGTTTTTTGCCACTGG------AATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
AGCCCCGTTTTTTGCCACTGGGTATTTAATTAAGCCAATAACGTCAAGTGCCAGGTCTAGTGAGTGGTAT
gnl|BL_ORD_ID|138 : evalue 5.72664e-16	138 at [1, 64, 17514, 17583]
AGCCCCGTTTTTTGCCACTGG------AATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
AGCCCCGTTTTTTGCCACTGGGTATTTAATTAAGCCAATAACGTCAAGTGCCAGGTCTAGTGAGTGGTAT
gnl|BL_ORD_ID|137 : evalue 5.72664e-16	137 at [1, 64, 17506, 17575]
AGCCCCGTTTTTTGCCACTGG------AATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
AGCCCCGTTTTTTGCCACTGGGTATTTAATTAAGCCAATAACGTCAAGTGCCAGGTCTAGTGAGTGGTAT
gnl|BL_ORD_ID|136 : evalue 5.72664e-16	136 at [1, 64, 17501, 17570]
AGCCCCGTTTTTTGCCACTGG

gnl|BL_ORD_ID|101 : evalue 5.72664e-16	101 at [1, 64, 17468, 17537]
AGCCCCGTTTTTTGCCACTGG------AATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
AGCCCCGTTTTTTGCCACTGGGTATTTAATTAAGCCAATAACGTCAAGTGCCAGGTCTAGTGAGTGGTAT
gnl|BL_ORD_ID|100 : evalue 5.72664e-16	100 at [1, 64, 17469, 17538]
AGCCCCGTTTTTTGCCACTGG------AATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
AGCCCCGTTTTTTGCCACTGGGTATTTAATTAAGCCAATAACGTCAAGTGCCAGGTCTAGTGAGTGGTAT
gnl|BL_ORD_ID|99 : evalue 5.72664e-16	99 at [1, 64, 17483, 17552]
AGCCCCGTTTTTTGCCACTGG------AATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
AGCCCCGTTTTTTGCCACTGGGTATTTAATTAAGCCAATAACGTCAAGTGCCAGGTCTAGTGAGTGGTAT
gnl|BL_ORD_ID|98 : evalue 5.72664e-16	98 at [1, 64, 17466, 17535]
AGCCCCGTTTTTTGCCACTGG------AATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
AGCCCCGTTTTTTGCCACTGGGTATTTAATTAAGCCAATAACGTCAAGTGCCAGGTCTAGTGAGTGGTAT
gnl|BL_ORD_ID|97 : evalue 5.72664e-16	97 at [1, 64, 17510, 17579]
AGCCCCGTTTTTTGCCACTGG------AATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
AGCCCCGTTTTTTGCCACTGGGTATTT

AGCCCCGTTTTTTGCCACTGGGTATTTAATTAAGCCAATAACGTCAAGTGCCAGGTCTAGTGAGTGGTAT
gnl|BL_ORD_ID|61 : evalue 5.72664e-16	61 at [1, 64, 17490, 17559]
AGCCCCGTTTTTTGCCACTGG------AATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
AGCCCCGTTTTTTGCCACTGGGTATTTAATTAAGCCAATAACGTCAAGTGCCAGGTCTAGTGAGTGGTAT
gnl|BL_ORD_ID|60 : evalue 5.72664e-16	60 at [1, 64, 17506, 17575]
AGCCCCGTTTTTTGCCACTGG------AATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
AGCCCCGTTTTTTGCCACTGGGTATTTAATTAAGCCAATAACGTCAAGTGCCAGGTCTAGTGAGTGGTAT
gnl|BL_ORD_ID|59 : evalue 5.72664e-16	59 at [1, 64, 17506, 17575]
AGCCCCGTTTTTTGCCACTGG------AATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
AGCCCCGTTTTTTGCCACTGGGTATTTAATTAAGCCAATAACGTCAAGTGCCAGGTCTAGTGAGTGGTAT
gnl|BL_ORD_ID|58 : evalue 5.72664e-16	58 at [1, 64, 17485, 17554]
AGCCCCGTTTTTTGCCACTGG------AATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT
AGCCCCGTTTTTTGCCACTGGGTATTTAATTAAGCCAATAACGTCAAGTGCCAGGTCTAGTGAGTGGTAT
gnl|BL_ORD_ID|57 : evalue 5.72664e-16	57 at [1, 64, 17390, 17459]
AGCCCCGTTTTTTGCCACTGG------AATT

[#<Bio::Blast::Report::Hit:0x0000000003b8bcf0 @hsps=[#<Bio::Blast::Report::Hsp:0x0000000003b7dd58 @hsp={}, @num=1, @bit_score=93.6635, @score=47, @evalue=9.63642e-21, @query_from=1, @query_to=64, @hit_from=17514, @hit_to=17583, @pattern_from=0, @pattern_to=0, @query_frame=1, @hit_frame=1, @identity=64, @positive=64, @gaps=6, @align_len=70, @density=0, @qseq="AGCCCCGTTTTTTGCCACTGG------AATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT", @hseq="AGCCCCGTTTTTTGCCACTGGTTATTTAATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT", @midline="|||||||||||||||||||||      |||||||||||||||||||||||||||||||||||||||||||">], @query_id="lcl|1_0", @query_def="myseq", @query_len=64, @num=1, @hit_id="gnl|BL_ORD_ID|154", @len=18959, @definition="NC_002549v1", @accession="154">, #<Bio::Blast::Report::Hit:0x0000000003b70220 @hsps=[#<Bio::Blast::Report::Hsp:0x0000000003b51eb0 @hsp={}, @num=1, @bit_score=93.6635, @score=47, @evalue=9.63642e-21, @query_from=1, @query_to=64, @hit_from=17514, @hit_to=17583, @pattern_from=0, @patte

#  Put together things that we have learned

## The Plan

* For each gene on the Arabidopsis gene list, we need the FASTA formatted sequence
* We can get this FASTA from DB fetch (we think!)
* We then index this FASTA file using makeblastdb to create a BLAST database
* We go for aperitivos to celebrate how smart we are :-)

##### unfortunately, things are seldom as easy as you think they will be!


**Your first task:  use dbfetch to retrieve the FASTA sequences for the <span style="color:red;">first 3-4 Arabidopsis gene IDs</span> on the list we have been using (NOT all of them, yet!)**  

* Go back in your course notes (or look-up using Google) the format for dbfetch URLs.  We need the ensemblgenomesgene database and fasta format.

* In your script, put all of the IDs into a single URL (i.e. don't call DBFetch over and over again)

* write the output to a file called "ARA.fa"

TASK 7A and TASK 7B are in separate Lecutre files.  Open them now and complete the tasks.

### be sure you understand Lesson 7 Task Solutions A and B before you continue!

# Accessing non-sequence data from NCBI - eUtils

Before we begin, take note of the following announcement from NCBI:

-----------
<em>Coming in May 2018: API Keys

On May 1, 2018, NCBI will begin enforcing the use of API keys that will offer enhanced levels of supported access to the E-utilities. After that date, any site (IP address) posting more than 3 requests per second to the E-utilities without an API key will receive an error message. By including an API key, a site can post up to 10 requests per second by default. Higher rates are available by request (vog.hin.mln.ibcn@seitilitue). Users can obtain an API key now from the Settings page of their NCBI account (to create an account, visit www.ncbi.nlm.nih.gov/account/). After creating the key, users should include it in each E-utility request by assigning it to the new api_key parameter.

Example request including an API key:
esummary.fcgi?db=pubmed&id=123456&api_key=ABCDE12345

Example error message if rates are exceeded:
{"error":"API rate limit exceeded","count":"11"}

Only one API key is allowed per NCBI account; however, a user may request a new key at any time. Such a request will invalidate any existing API key associated with that NCBI account.

We encourage regular E-utility users to obtain an API key as soon as possible and begin the process of incorporating it into code. We also encourage users to monitor their request rates to determine if they will require rates higher than 10 per second. As stated above, we can potentially have higher rates negotiated prior to the beginning of enforcement on May 1, 2018.</em>

----------------


We may need to have all students register with NCBI; however, I will try to avoid this by providing examples that make only single requests.  


# OVERVIEW 

**The Nine E-utilities in Brief**

EUtilities consists of 9 "REST" style interfaces (i.e. parameterized URLs):


* **EInfo** (database statistics)

    http://eutils.ncbi.nlm.nih.gov/entrez/eutils/einfo.fcgi

    Provides the number of records indexed in each field of a given database, the date of the last update of the database, and the available links from the database to other Entrez databases.

* **ESearch** (text searches)

    http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi

    Responds to a text query with the list of matching UIDs in a given database (for later use in ESummary, EFetch or ELink), along with the term translations of the query.

* **EPost** (UID uploads)

    http://eutils.ncbi.nlm.nih.gov/entrez/eutils/epost.fcgi

    Accepts a list of UIDs from a given database, stores the set on the History Server, and responds with a query key and web environment for the uploaded dataset.

* **ESummary** (document summary downloads)

    http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi

    Responds to a list of UIDs from a given database with the corresponding document summaries.

* **EFetch** (data record downloads)

    http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi

    Responds to a list of UIDs in a given database with the corresponding data records in a specified format.

* **ELink** (Entrez links)

    http://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi

    Responds to a list of UIDs in a given database with either a list of related UIDs (and relevancy scores) in the same database or a list of linked UIDs in another Entrez database; checks for the existence of a specified link from a list of one or more UIDs; creates a hyperlink to the primary LinkOut provider for a specific UID and database, or lists LinkOut URLs and attributes for multiple UIDs.

* **EGQuery** (global query)

    http://eutils.ncbi.nlm.nih.gov/entrez/eutils/egquery.fcgi

    Responds to a text query with the number of records matching the query in each Entrez database.

* **ESpell** (spelling suggestions)

    http://eutils.ncbi.nlm.nih.gov/entrez/eutils/espell.fcgi

    Retrieves spelling suggestions for a text query in a given database.

* **ECitMatch** (batch citation searching in PubMed)

    http://eutils.ncbi.nlm.nih.gov/entrez/eutils/ecitmatch.cgi

    Retrieves PubMed IDs (PMIDs) corresponding to a set of input citation strings.




# Getting Started with EUtils

Before we look at the BioRuby Classes for EUtils, we will look at the EUtils REST interface.  The complete tutorial for EUtils is at https://www.ncbi.nlm.nih.gov/books/NBK25497/

I will take you through a (small!) subset of these functionalities in class.  With that background, you should be able to understand the rest of the functionality yourself.

To begin, click on the link for EInfo:


http://eutils.ncbi.nlm.nih.gov/entrez/eutils/einfo.fcgi

This lists all of the databases that you can query/access from NCBI.  Each of those databases uses a **different type of identifier.**  Those are listed [here](https://www.ncbi.nlm.nih.gov/books/NBK25497/table/chapter2.T._entrez_unique_identifiers_ui/?report=objectonly)

To find out more about any of those databases (in particular, the fields that can be queried, and the database-to-database linkages that are available!) the pattern is like what we learned for dbFetch - a set of key/value pairs after a '?' (separated by ';' if there is more than one)

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/einfo.fcgi?db=nuccore


From the output of nuccore we can see, for example, that you can query the field "Title" (TITL) in the nuccore database:

    <Field>
        <Name>TITL</Name>
        <FullName>Title</FullName>


The important part of that is the field name:  **TITL** and we will now see how to use it...


# Searching NCBI Databases


NCBI databases use a query language called ENTREZ.  ENTREZ queries are structured as "value"[FIELD]

To do queries on the, for example, the Nucleotide database ('nuccore'), you would use the ESearch utility.  If you were looking for nucleotide entries with titles including the phrase "apoptosis inhibitor", you might create a URL with the following structure:

[http://eutils.ncbi.nlm.nih.gov/entrez/eutils/**esearch.fcgi?db=nuccore;term="apoptosis+inhibitor"[TITL]**](http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nuccore;term="apoptosis+inhibitor"[TITL])

The output of that call is a list of identifiers (what kind of identifiers?  See the "nuccore" entry in the linked table above) for the first 20 records that have "apoptosis inhibitor" in their title.

<code>

    <eSearchResult>
    <Count>2138</Count>   <----   The number of matches in the database
    <RetMax>20</RetMax>   <----   The maximum number of records returned
    <RetStart>0</RetStart>  <---- The "offset" - i.e. what record am I starting with?  (in this case the first)
    <IdList>
    <Id>219283154</Id><Id>219283151</Id><Id>219283149</Id><Id>197098447</Id><Id>148230698</Id><Id>148229867</Id><Id>57529973</Id><Id>1279832269</Id><Id>1279832267</Id><Id>1279831517</Id><Id>1279711068</Id><Id>1279105385</Id><Id>1279090872</Id><Id>1279061012</Id><Id>1279061010</Id><Id>1279048499</Id><Id>1279048498</Id><Id>1279048496</Id><Id>1279048494</Id><Id>816197788</Id>
    </IdList>
    <TranslationSet/>
    <TranslationStack>
    <TermSet>
    <Term>"apoptosis inhibitor"[TITL]</Term>
    <Field>TITL</Field>
    <Count>2138</Count>
    <Explode>N</Explode>
    </TermSet>
    <OP>GROUP</OP>
    </TranslationStack>
    <QueryTranslation>"apoptosis inhibitor"[TITL]</QueryTranslation>
    </eSearchResult>

</code>

To get more results, you change/add parameters.  For example, to get the next 50 records AFTER these records, the URL would be:


[http://eutils.ncbi.nlm.nih.gov/entrez/eutils/**esearch.fcgi?<span style="color:red;">retstart=20;retmax=50</span>;db=nuccore;term="apoptosis+inhibitor"[TITL]**](http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?retstart=20;retmax=50;db=nuccore;term="apoptosis+inhibitor"[TITL])

Finally, to increase the complexity of the search, you use AND, OR, and NOT operators, with the same "value"[FIELD] structure.  So to limit the current query to only those from the genus "Spodoptera" ("ORGN = Organism) (NOTE that we reset the retstart back to zero!!)

[http://eutils.ncbi.nlm.nih.gov/entrez/eutils/**esearch.fcgi?retstart=0;retmax=50;db=nuccore;<span style="color:red;">term="apoptosis+inhibitor"[TITL]+AND+"Spodoptera"[ORGN]**</span>](http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?retstart=0;retmax=50;db=nuccore;term="apoptosis+inhibitor"[TITL]+AND+"Spodoptera"[ORGN])

The result is a single DNA sequence:  gi:1274128812


# Retrieving from search results

To retrieve that sequence, we use the EFetch utility ([Documentation Here](https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.EFetch)).  The process is exactly the same - create URL parameters after <code>efetch.fcgi?</code>  The two parameters of greatest importance to us are "db=" (which database am I fetching from?) and "id=" (which records am I fetching?).  So our next URL looks like this:

[http://eutils.ncbi.nlm.nih.gov/entrez/eutils/**efetch.fcgi?db=nuccore;id=1274128812**](http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore;id=1274128812)

Note:  if you click that, it will pop-up a window.  That is because the default data-type returned from the Nucleotide sequence database (nuccore) is called "ASN.1", and you probably don't have a utility that will read that format.  You **do** have a utility that will read GenBank format, though!  **BioRuby!!**   So is there a way to get that record in GenBank format?  Yes.  There are two additional parameters for EFetch:  retmode and rettype.  Each of the databases supports a subset of possible values for retmode and rettype.  You can look at the possibilities [HERE](https://www.ncbi.nlm.nih.gov/books/NBK25499/table/chapter4.T._valid_values_of__retmode_and/?report=objectonly).  For us to get a GenBank formatted file, we have to set retmode=gb and rettype=text:

[http://eutils.ncbi.nlm.nih.gov/entrez/eutils/**efetch.fcgi?db=nuccore;id=1274128812;<span style="color:red;">rettype=gb;retmode=text**</span>](http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore;rettype=gb;retmode=text;id=1274128812)

Of course, if you need more than one record, you can request them all at the same time by adding more GI numbers separated by commas:


[http://eutils.ncbi.nlm.nih.gov/entrez/eutils/**efetch.fcgi?db=nuccore;rettype=gb;retmode=text;<span style="color:red;">id=1274128812,1274128812,1274128812,1274128812**</span>](http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore;rettype=gb;retmode=text;id=1274128812,1274128812,1274128812,1274128812)

From this point, you know what to do!  This is exactly the same as what we learned with EFetch and using BioRuby to parse the output.

**BE CAREFUL!!**   NCBI is very very strict about their limits on number of requests!  For an account with no "license", you can only call 3 times per second.  With a license, only 10 times per second!  **So do NOT do this:**

   

    ids.each do |id|
     one_record = fetch("http://eutils.../efetch.fcgi?db=nuccore;rettype=gb;retmode=text;id=#{id}")
     process(one_record)
    end

 

**That is how you get your entire institute BANNED from using NCBI utilities!**  

<span style="color:red;">YOU HAVE BEEN WARNED!!  :-)</span>



# Prove that you understand

* Find the PubMed publications written by an author named Mark D Wilkinson who studies "Reproducibility of Results" (find the [MESH keyword](https://meshb.nlm.nih.gov/search) and use it in the [MESH] field of the search query.  Retrieve the abstract for the first 3 of these, in text format.

(You will need to search for how ENTREZ represents author names!)


# Enough of this REST stuff, let's use BioRuby!

The BioRuby interface is very "thin" - it simply makes it easier to construct the EUtilities REST URLs.

http://bioruby.org/rdoc/Bio/NCBI/REST


A typical call is like this:


In [4]:
require 'bio'

ncbi = Bio::NCBI::REST.new
max = 5
ids = ncbi.esearch('"Wilkinson MD"[AUTH]', {"db"=>"nucleotide", "rettype"=>"gb", "retmax" => max})
idlist = ids.join(",")
puts idlist
puts "\n\nRecords are\n\n"
records = ncbi.efetch(idlist, {"db"=>"nucleotide", "rettype"=>"gb", "retmode"=>"text", "retmax"=> max})
puts records

1676439675,1388704174,33527965,33527963,33527961


Records are


LOCUS       NM_018156              16282 bp    mRNA    linear   PRI 21-AUG-2019
DEFINITION  Homo sapiens vacuolar protein sorting 13 homolog D (VPS13D),
            transcript variant 2, mRNA.
ACCESSION   NM_018156
VERSION     NM_018156.4
KEYWORDS    RefSeq.
SOURCE      Homo sapiens (human)
  ORGANISM  Homo sapiens
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;
            Catarrhini; Hominidae; Homo.
REFERENCE   1  (bases 1 to 16282)
  AUTHORS   Seong E, Insolera R, Dulovic M, Kamsteeg EJ, Trinh J, Bruggemann N,
            Sandford E, Li S, Ozel AB, Li JZ, Jewett T, Kievit AJA, Munchau A,
            Shakkottai V, Klein C, Collins CA, Lohmann K, van de Warrenburg BP
            and Burmeister M.
  TITLE     Mutations in VPS13D lead to a new recessive ataxia with spasticity
            and mitochondrial defects
  JOURN

# Better use of PubMed

Most reference managers (EndNote, Mendeley, and even LaTex) are able to work with files in BibTex format.  Let's do one last BioRuby call to PubMed to retrieve articles in that format.

Our task is to get the abstracts for the nucleotide sequence files that were discovered in the search above.  This is called a "Link".  Earlier in this Lesson, we looked at the output from the EInfo tool - it told you what FIELDs you could query for each database, but also what LINKS existed between one database and another.  Let's look at that again - the EInfo about the nuccore database:

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/einfo.fcgi?db=nuccore

Near the end of that page you will find:

    <Link>
        <Name>nuccore_pubmed</Name>
        <Menu>PubMed Links</Menu>
        <Description>PubMed articles cited by Nucleotide sequence record</Description>
        <DbTo>pubmed</DbTo>
    </Link>

This describes that there is a link between nuccore (the sequence database) and pubmed (the publications database).  The tool that we need to find the links between the list of IDs for Database A and the list of IDs for Database B is the ELink tool

http://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi

It works like all other EUtils; however, there is no BioRuby wrapper for this one, so we must do the hard work ourselves.  To see some example code in Perl go to:  https://www.ncbi.nlm.nih.gov/books/NBK25498/#chapter3.Application_4_Finding_unique_se.  We will now do the same thing in Ruby.


### ELink parameters are:  
* dbfrom
* db  (i.e. which db are you linking TO)
* id = ...list of ids...

So, like all other EUtils, we can construct a URL that calls the ELink service:

http://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=nuccore;db=pubmed;id=732693005,732693505,1129728243,1129728241,1129728239

If you click on that, you will see the XML document that contains the PubMed IDs that correspond to the nucleotide sequence IDs we sent to the **elink** tool.  They are separated into "Link Sets", corresponding to different subsections of pubmed (e.g. **nuccore_pubmed_refseq**).  

HOWEVER:  If you read a little bit further in the documentation (for all EUtilities!), the instructions say "use POST when you have many IDs (>200)" 

>**Required Parameter – Used only when input is from a UID list**

>**id**

>UID list. Either a single UID or a comma-delimited list of UIDs may be provided. All of the UIDs must be from the database specified by dbfrom. There is no set maximum for the number of UIDs that can be passed to ELink, but **if more than about 200 UIDs are to be provided, the request should be made using the HTTP POST method.**


What does "HTTP POST" mean?   We will discuss it in detail in a later lecture, but for now, you only need to understand that it is another way of sending information to a Web server.  Using HTTP GET, you put the information at the end of the URL (<code>scriptname?a=b;c=d;...</code>).   With HTTP POST you send it in the "body" of a message.  

It is important to learn how to create an HTTP POST request, so even though we _can_ use the URL above (because we only have 5 ID numbers), we should look at and understand the other solution.  An example of an HTTP POST is below, where we are sending a request to the ELink service of NCBI:



In [5]:
require 'net/http'
require 'bio'

ncbi = Bio::NCBI::REST.new
max = 5
ids = ncbi.esearch('"Wilkinson MD"[AUTH]', {"db"=>"nucleotide", "rettype"=>"gb", "retmax" => max})
idlist = ids.join(",")
puts idlist

# prepare for HTTP POST - this is very similar to what is in the "fetch" routine from your other scripts.

elink = URI("https://eutils.ncbi.nlm.nih.gov:443/entrez/eutils/elink.fcgi")
request = Net::HTTP::Post.new(elink)
https = Net::HTTP.new(elink.host,elink.port)
https.use_ssl = true  # tell the HTTP connection to use ecryption (SSL = Secure Socket Layer)

# here is where we create the content of the body

request.set_form_data(
  'email' => 'markw@illuminae.com',
  'dbfrom' => 'nuccore', 
  'db' => 'pubmed',
  'retmode' => 'xml',
  'id' => idlist)

output = https.request(request)   # make the request (we really should check if it is successful)
puts output.body

1676439675,1388704174,33527965,33527963,33527961
<?xml version="1.0" encoding="UTF-8" ?>
<!DOCTYPE eLinkResult PUBLIC "-//NLM//DTD elink 20101123//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20101123/elink.dtd">
<eLinkResult>

  <LinkSet>
    <DbFrom>nuccore</DbFrom>
    <IdList>
      <Id>1676439675</Id>
      <Id>1388704174</Id>
      <Id>33527965</Id>
      <Id>33527963</Id>
      <Id>33527961</Id>
    </IdList>
    <LinkSetDb>
      <DbTo>pubmed</DbTo>
      <LinkName>nuccore_pubmed</LinkName>
      
        <Link>
				<Id>30789691</Id>
			</Link>
        <Link>
				<Id>29604224</Id>
			</Link>
        <Link>
				<Id>29518281</Id>
			</Link>
        <Link>
				<Id>29307555</Id>
			</Link>
        <Link>
				<Id>25896417</Id>
			</Link>
        <Link>
				<Id>20610895</Id>
			</Link>
        <Link>
				<Id>20301317</Id>
			</Link>
        <Link>
				<Id>19056867</Id>
			</Link>
        <Link>
				<Id>15498460</Id>
			</Link>
        <Link>
				<Id>15070428</Id>
			</Link>
      


#  Parsing XML

In an earlier lesson we learned how to extract information from JSON.  In the example above the output of the EUtils service is in XML format, so we now need to learn how to extract information from XML.

The most popular Ruby library for XML parsing is called 'nokogiri' (this translates to English as "saw" and Spanish as "sierra" como "sierra circular").  So that's what we will use.  The XML output from EUtils is quite simple (we will see more complex examples later in the course!) so let's use it as our learning example


    <eLinkResult>
        <LinkSet>
            <LinkSetDb>
                  <DbTo>pubmed</DbTo>
                  <LinkName>nuccore_pubmed</LinkName>

                  <Link>
                        <Id>25896417</Id>
                  </Link>
                  <Link>
                        <Id>20610895</Id>
                  </Link>
                  <Link>
                        <Id>19056867</Id>
                  </Link>
                  <Link>
                        <Id>15498460</Id>
                  </Link>
                  <Link>
                        <Id>12421765</Id>
                  </Link>

            </LinkSetDb>
        </LinkSet>
     <eLinkResut>

This is the relevant section we want to extract IDs from.  XML documents are "hierarchical", sub-sections are enclosed in parent sections in the pattern:
        
        <A> <B> data </B>  </A>

Given this structure, it is normal to talk about the "path" to the data.  In this case, the data is A-->B-->data.  

In the output from ELink, the path is:  LinkSetDb-->Link-->Id-->data

These "paths" are used in Ruby to define the piece of data we want to extract.  For example:



In [6]:
require 'nokogiri'

xml = <<EOF
    <eLinkResult>
        <LinkSet>

            <LinkSetDb>
                <Link>
                      <Id>25896417</Id>
                </Link>
                <Link>
                      <Id>20610895</Id>
                </Link>
            </LinkSetDb>
        </LinkSet>
    <eLinkResut>
EOF

doc = Nokogiri::XML(xml)
doc.xpath('//LinkSet/LinkSetDb/Link').each do |node|    # the "path" to the data, starts with '//'
  puts "PubMed = " + node.at_xpath('Id').content
end

PubMed = 25896417
PubMed = 20610895


[#<Nokogiri::XML::Element:0x1ea765c name="Link" children=[#<Nokogiri::XML::Text:0x1eac3a0 "\n                      ">, #<Nokogiri::XML::Element:0x1ea7454 name="Id" children=[#<Nokogiri::XML::Text:0x1eadf5c "25896417">]>, #<Nokogiri::XML::Text:0x1eadd18 "\n                ">]>, #<Nokogiri::XML::Element:0x1ea69b4 name="Link" children=[#<Nokogiri::XML::Text:0x1ead994 "\n                      ">, #<Nokogiri::XML::Element:0x1ea6860 name="Id" children=[#<Nokogiri::XML::Text:0x1ead714 "20610895">]>, #<Nokogiri::XML::Text:0x1ead570 "\n                ">]>]

#  Putting it all together!

Finally, we will create a script that:

* retrieves the nucleotide database ids
* retrieves the cross-reference links to PubMed
* extracts the PubMed IDs
* retrieves the PubMed Abstracts for those IDs


In [7]:
require 'net/http'
require 'bio'
require 'nokogiri'

ncbi = Bio::NCBI::REST.new
max = 5
ids = ncbi.esearch('"Wilkinson MD"[AUTH]', {"db"=>"nucleotide", "rettype"=>"gb", "retmax" => max})
idlist = ids.join(",")

elink = URI("https://eutils.ncbi.nlm.nih.gov:443/entrez/eutils/elink.fcgi")
request = Net::HTTP::Post.new(elink)
https = Net::HTTP.new(elink.host,elink.port)
https.use_ssl = true  # tell the HTTP connection to use ecryption (SSL = Secure Socket Layer)

request.set_form_data(
  'email' => 'markw@illuminae.com',
  'dbfrom' => 'nuccore', 
  'db' => 'pubmed',
  'retmode' => 'xml',
  'id' => idlist)

output = https.request(request)   # make the request (we really should check if it is successful)
linkset = output.body

doc = Nokogiri::XML(linkset)
pubmedids = []
doc.xpath('//LinkSet/LinkSetDb/Link').each do |node|    # the "path" ("xpath")to the data, starts with '//'
   pubmedids << node.at_xpath('Id').content.to_s
end
puts "PUBMED IDs are: #{pubmedids}"

pubmedrecords = ncbi.efetch(pubmedids, {"db"=>"pubmed", "rettype"=>"abstract", "retmode"=>"text"})
puts pubmedrecords.force_encoding("utf-8")  #This is a useful trick for text that has international characters!

PUBMED IDs are: ["30789691", "29604224", "29518281", "29307555", "25896417", "20610895", "20301317", "19056867", "15498460", "15070428", "12421765", "30789691", "30258100", "29604224", "29568061", "29518281", "29507755", "29307555", "26344197", "25896417", "21873635", "21182205", "20610895", "20379614", "20301317", "19056867", "16710414", "15498460", "15489334", "14702039", "12477932", "12421765", "9455484", "9110174", "8619474", "30789691", "30258100", "29604224", "29568061", "29518281", "29507755", "29307555", "26344197", "25896417", "21873635", "21182205", "20610895", "20379614", "20301317", "19056867", "16710414", "15498460", "15489334", "14702039", "14681893", "12477932", "12421765", "11960835", "9455484", "9110174", "8619474"]
1. VPS13D Movement Disorder.

Meijer IA(1).
In: Adam MP, Ardinger HH, Pagon RA, Wallace SE, Bean LJH, Stephens K, Amemiya A, 
editors. GeneReviews® [Internet]. Seattle (WA): University of Washington,
Seattle; 1993-2019.
2019 Feb 21.

Author information: 
(1


# Bio::PubMed , Bio::MEDLINE and bibtex format

The output above is great for a human, but not very useful for a machine.  Most reference managers can accept new records in bibtex format.  Can we get bibtex format for these records?  Sure!

Bio::PubMed is a shortcut to the ESearch and EFetch of utilities for the pubmed database.  It is used like this:



In [8]:
pubmedids = ["25896417", "20610895", "19056867"]

Bio::PubMed.efetch(pubmedids).each do |pubmedrecord|
  # pubmedrecord is a string of the record, in pubmed format
  # we can use that to create a new Bio::MEDLINE object
  medline = Bio::MEDLINE.new(pubmedrecord)
  reference = medline.reference  # reference is a Bio::Reference object
  puts reference.bibtex   # the .bibtex method creates the bibtex string of the Bio::Reference
end

@article{PMID:25896417,
  author       = {Nakada, T. A. and Boyd, J. H. and Russell, J. A. and Aguirre-Hernandez, R. and Wilkinson, M. D. and Thair, S. A. and Nakada, E. and McConechy, M. K. and Fjell, C. D. and Walley, K. R.},
  title        = {VPS13D Gene Variant Is Associated with Altered IL-6 Production and Mortality in Septic Shock.},
  journal      = {J Innate Immun},
  year         = {2015},
  volume       = {7},
  number       = {5},
  pages        = {545--553},
  url          = {http://www.ncbi.nlm.nih.gov/pubmed/25896417},
}

@article{PMID:20610895,
  author       = {Koriyama, H. and Nakagami, H. and Katsuya, T. and Sugimoto, K. and Yamashita, H. and Takami, Y. and Maeda, S. and Kubo, M. and Takahashi, A. and Nakamura, Y. and Ogihara, T. and Rakugi, H. and Kaneda, Y. and Morishita, R.},
  title        = {Identification of evidence suggestive of an association with peripheral arterial disease at the OSBPL10 locus by genome-wide investigation in the Japanese population.},
  jou

["PMID- 25896417\nOWN - NLM\nSTAT- MEDLINE\nDCOM- 20160620\nLR  - 20150828\nIS  - 1662-8128 (Electronic)\nIS  - 1662-811X (Linking)\nVI  - 7\nIP  - 5\nDP  - 2015\nTI  - VPS13D Gene Variant Is Associated with Altered IL-6 Production and Mortality in\n      Septic Shock.\nPG  - 545-53\nLID - 10.1159/000381265 [doi]\nAB  - BACKGROUND: Genetic variations contribute to septic shock mortality. To discover \n      a novel locus, we performed in vitro genome-wide association studies (GWAS) and\n      further tested the result in a cohort of septic shock patients. METHODS: Two in\n      vitro GWAS using a quantitative trait locus analysis of stimulated IL-6\n      production in lymphoblastoid cells from 60 individuals of European ancestry were \n      performed. VPS13D rs6685273 was genotyped in European ancestry patients (n =\n      498). The VPS13D gene was silenced in vitro. RESULTS: Two GWAS using\n      lymphoblastoid cells identified the locus of VPS13D rs6685273 that was\n      significa

<pre>
  

</pre>
  
# The Final Code

So finally we put the BibTex code at the end of our script, and...


In [10]:
require 'net/http'
require 'bio'
require 'nokogiri'


ncbi = Bio::NCBI::REST.new
max = 5
ids = ncbi.esearch('"Wilkinson MD"[AUTH]', {"db"=>"nucleotide", "rettype"=>"gb", "retmax" => max})
idlist = ids.join(",")

elink = URI("https://eutils.ncbi.nlm.nih.gov:443/entrez/eutils/elink.fcgi")
request = Net::HTTP::Post.new(elink)
https = Net::HTTP.new(elink.host,elink.port)
https.use_ssl = true  # tell the HTTP connection to use ecryption (SSL = Secure Socket Layer)

request.set_form_data(
  'email' => 'markw@illuminae.com',
  'dbfrom' => 'nuccore', 
  'db' => 'pubmed',
  'retmode' => 'xml',
  'id' => idlist)

output = https.request(request)   # make the request (we really should check if it is successful)
linkset = output.body

doc = Nokogiri::XML(linkset)
pubmedids = []  # force it to be a string so that we can use '<<' operator
doc.xpath('//LinkSet/LinkSetDb/Link').each do |node|    # the "path" ("xpath")to the data, starts with '//'
   pubmedids << node.at_xpath('Id').content.to_s
end

Bio::PubMed.efetch(pubmedids).each do |pubmedrecord|
  # pubmedrecord is a string of the record, in pubmed format
  # we can use that to create a new Bio::MEDLINE object
  medline = Bio::MEDLINE.new(pubmedrecord)
  reference = medline.reference  # reference is a Bio::Reference object
  puts reference.bibtex   # the .bibtex method creates the bibtex string of the Bio::Reference
end

@article{PMID:30789691,
  author       = {Meijer, I. A.},
  title        = {VPS13D Movement Disorder},
  journal      = {},
  year         = {1993},
  volume       = {},
  url          = {http://www.ncbi.nlm.nih.gov/pubmed/30789691},
}

@article{PMID:29604224,
  author       = {Seong, E. and Insolera, R. and Dulovic, M. and Kamsteeg, E. J. and Trinh, J. and Bruggemann, N. and Sandford, E. and Li, S. and Ozel, A. B. and Li, J. Z. and Jewett, T. and Kievit, A. J. A. and Munchau, A. and Shakkottai, V. and Klein, C. and Collins, C. A. and Lohmann, K. and van de Warrenburg, B. P. and Burmeister, M.},
  title        = {Mutations in VPS13D lead to a new recessive ataxia with spasticity and mitochondrial defects.},
  journal      = {Ann Neurol},
  year         = {2018},
  volume       = {83},
  number       = {6},
  pages        = {1075--1088},
  url          = {http://www.ncbi.nlm.nih.gov/pubmed/29604224},
}

@article{PMID:29518281,
  author       = {Gauthier, J. and Meijer, I. A. and Lessel,

@article{PMID:29307555,
  author       = {Anding, A. L. and Wang, C. and Chang, T. K. and Sliter, D. A. and Powers, C. M. and Hofmann, K. and Youle, R. J. and Baehrecke, E. H.},
  title        = {Vps13D Encodes a Ubiquitin-Binding Protein that Is Required for the Regulation of Mitochondrial Size and Clearance.},
  journal      = {Curr Biol},
  year         = {2018},
  volume       = {28},
  number       = {2},
  pages        = {287--295.e6},
  url          = {http://www.ncbi.nlm.nih.gov/pubmed/29307555},
}

@article{PMID:26344197,
  author       = {Wan, C. and Borgeson, B. and Phanse, S. and Tu, F. and Drew, K. and Clark, G. and Xiong, X. and Kagan, O. and Kwan, J. and Bezginov, A. and Chessman, K. and Pal, S. and Cromar, G. and Papoulas, O. and Ni, Z. and Boutz, D. R. and Stoilova, S. and Havugimana, P. C. and Guo, X. and Malty, R. H. and Sarov, M. and Greenblatt, J. and Babu, M. and Derry, W. B. and Tillier, E. R. and Wallingford, J. B. and Parkinson, J. and Marcotte, E. M. and Emili

@article{PMID:15489334,
  author       = {Gerhard, D. S. and Wagner, L. and Feingold, E. A. and Shenmen, C. M. and Grouse, L. H. and Schuler, G. and Klein, S. L. and Old, S. and Rasooly, R. and Good, P. and Guyer, M. and Peck, A. M. and Derge, J. G. and Lipman, D. and Collins, F. S. and Jang, W. and Sherry, S. and Feolo, M. and Misquitta, L. and Lee, E. and Rotmistrovsky, K. and Greenhut, S. F. and Schaefer, C. F. and Buetow, K. and Bonner, T. I. and Haussler, D. and Kent, J. and Kiekhaus, M. and Furey, T. and Brent, M. and Prange, C. and Schreiber, K. and Shapiro, N. and Bhat, N. K. and Hopkins, R. F. and Hsie, F. and Driscoll, T. and Soares, M. B. and Casavant, T. L. and Scheetz, T. E. and Brown-stein, M. J. and Usdin, T. B. and Toshiyuki, S. and Carninci, P. and Piao, Y. and Dudekula, D. B. and Ko, M. S. and Kawakami, K. and Suzuki, Y. and Sugano, S. and Gruber, C. E. and Smith, M. R. and Simmons, B. and Moore, T. and Waterman, R. and Johnson, S. L. and Ruan, Y. and Wei, C. L. and M

@article{PMID:8619474,
  author       = {Andersson, B. and Wentland, M. A. and Ricafrente, J. Y. and Liu, W. and Gibbs, R. A.},
  title        = {A "double adaptor" method for improved shotgun library construction.},
  journal      = {Anal Biochem},
  year         = {1996},
  volume       = {236},
  number       = {1},
  pages        = {107--113},
  url          = {http://www.ncbi.nlm.nih.gov/pubmed/8619474},
}

@article{PMID:30789691,
  author       = {Meijer, I. A.},
  title        = {VPS13D Movement Disorder},
  journal      = {},
  year         = {1993},
  volume       = {},
  url          = {http://www.ncbi.nlm.nih.gov/pubmed/30789691},
}

@article{PMID:30258100,
  author       = {Conte, C. and Griffis, E. R. and Hickson, I. and Perez-Oliva, A. B.},
  title        = {USP45 and Spindly are part of the same complex implicated in cell migration.},
  journal      = {Sci Rep},
  year         = {2018},
  volume       = {8},
  number       = {1},
  pages        = {14375},
  url          

@article{PMID:15498460,
  author       = {Velayos-Baeza, A. and Vettori, A. and Copley, R. R. and Dobson-Stone, C. and Monaco, A. P.},
  title        = {Analysis of the human VPS13 gene family.},
  journal      = {Genomics},
  year         = {2004},
  volume       = {84},
  number       = {3},
  pages        = {536--549},
  url          = {http://www.ncbi.nlm.nih.gov/pubmed/15498460},
}

@article{PMID:15489334,
  author       = {Gerhard, D. S. and Wagner, L. and Feingold, E. A. and Shenmen, C. M. and Grouse, L. H. and Schuler, G. and Klein, S. L. and Old, S. and Rasooly, R. and Good, P. and Guyer, M. and Peck, A. M. and Derge, J. G. and Lipman, D. and Collins, F. S. and Jang, W. and Sherry, S. and Feolo, M. and Misquitta, L. and Lee, E. and Rotmistrovsky, K. and Greenhut, S. F. and Schaefer, C. F. and Buetow, K. and Bonner, T. I. and Haussler, D. and Kent, J. and Kiekhaus, M. and Furey, T. and Brent, M. and Prange, C. and Schreiber, K. and Shapiro, N. and Bhat, N. K. and Hopkins, R. F.

@article{PMID:11960835,
  author       = {Swartz, B. E. and Burmeister, M. and Somers, J. T. and Rottach, K. G. and Bespalova, I. N. and Leigh, R. J.},
  title        = {A form of inherited cerebellar ataxia with saccadic intrusions, increased saccadic speed, sensory neuropathy, and myoclonus.},
  journal      = {Ann N Y Acad Sci},
  year         = {2002},
  volume       = {956},
  pages        = {441--444},
  url          = {http://www.ncbi.nlm.nih.gov/pubmed/11960835},
}

@article{PMID:9455484,
  author       = {Seki, N. and Ohira, M. and Nagase, T. and Ishikawa, K. and Miyajima, N. and Nakajima, D. and Nomura, N. and Ohara, O.},
  title        = {Characterization of cDNA clones in size-fractionated cDNA libraries from human brain.},
  journal      = {DNA Res},
  year         = {1997},
  volume       = {4},
  number       = {5},
  pages        = {345--349},
  url          = {http://www.ncbi.nlm.nih.gov/pubmed/9455484},
}

@article{PMID:9110174,
  author       = {Yu, W. and Andersson,

["PMID- 30789691\nSTAT- Publisher\nDA  - 20190222\nCTDT- 20190221\nPB  - University of Washington, Seattle\nDP  - 1993\nTI  - VPS13D Movement Disorder\nBTI - GeneReviews((R))\nAB  - CLINICAL CHARACTERISTICS: VPS13D movement disorder is a hyperkinetic movement\n      disorder (dystonia, chorea, and/or ataxia) of variable age of onset that can be\n      associated with developmental delay. Onset ranges from birth to adulthood.\n      Individuals can present in childhood with motor delays and gait instability.\n      Cognitive impairment ranging from mild intellectual disability to developmental\n      delay has been reported, and several individuals have normal cognitive function. \n      Individuals have also presented as young adults with gait difficulties caused by \n      spastic ataxia or ataxia. In addition to gait ataxia, affected individuals had\n      limb ataxia, dysarthria, and eye movement abnormalities (macro-saccadic\n      oscillations, nystagmus, and saccadic pursuit). Ad

...and you can write that to a file and import it into EndNote, or Mendeley, or PaperPile, or whatever reference manager you prefer!


DONE!
